In [ ]:
(ql:quickload :kenzo)

In [ ]:
(in-package "CAT")

In [ ]:
(cat-init)

# Coalgebras and cobars

## Introduction

The implementation of coalgebras and cobars follows closely the section 6 of the paper
*Effective Homology: a survey* (Available
at the web site at the web site at the web site at the web site [**https://www-fourier.ujf.grenoble.fr/~sergerar/**](https://www-fourier.ujf.grenoble.fr/~sergerar/)).

## Coalgebras

A coalgebra  is a pair $(C, \chi)$ where
1. $C$ is a chain complex.
2. $\chi$ (the *coproduct*) is a chain complex morphism $$\chi: C \rightarrow C \otimes C.$$

These components must satisfy the usual structural properties of the coalgebras (**Mac Lane**, Homology, section VI-9. Springer 1970.).

### Implementation of a coalgebra

A coalgebra is represented as an instance of the CLOS class ``COALGEBRA``, subclass of the ``CHAIN COMPLEX`` class. The definition of this new class is simply:

```lisp
(DEFCLASS COALGEBRA (chain-complex)
    ((cprd :type morphism :initarg :cprd :reader cprd1)))   ;; coproduct
```

So, this new class inherits the slots of the ``CHAIN COMPLEX`` class and has a slot of its own, namely ``cprd``, a morphism  object representing the coproduct. The user will note the important following fact that, **once a coalgebra has been defined, one may use on it any
function or method applicable to a chain complex**.

### The function ``build-clgb``

To facilitate the construction of instances of the ``COALGEBRA`` class, the software provides the function ``build-clgb``,

```lisp
(build-clgb :cmpr cmpr :basis basis :bsgn bsgn :intr-dffr intr-dffr :dffr-strt dffr-strt :intr-cprd intr-cprd :cprd-strt cprd-strt :orgn orgn)
```

defined with keyword parameters. The returned value is an instance of type ``COALGEBRA``. The keyword arguments are:

- *cmpr*, a comparison function for generators.
- *basis*, the basis function for the underlying chain complex.
- *bsgn*, the  base point, a generator of any type.
- *intr-dffr*, the differential lisp function for the chain complex.
- *dffr-strt*, the strategy (``:gnrt`` or ``:cmbn``) for the differential.
- *intr-cprd*, the lisp function for the coproduct of the coalgebra.
- *cprd-strt*, the strategy (``:gnrt`` or ``:cmbn``) for the coproduct.
- *orgn*, an adequate comment list.

The function ``build-clgb`` calls  the function ``build-chcm`` and with the help of the function
``build-mrph`` builds the coproduct morphism of degree 0, to settle the slot ``crpd`` of the instance. According to the definition, the source slot of the coproduct morphism is the underlying chain complex and the target slot is the tensor product of this chain complex with itself. As to the other objects,
an identification number $n$ (slot ``idnm``) is assigned to this Kenzo object and the ``COALGEBRA`` instance is  pushed  onto the list ``*clgb-list*``. The  associated printing method prints a string like ``[K``*n* ``Coalgebra]``.



### Miscellaneous functions and macros for coalgebras

``(cat-init)`` *[Function]*
> Clear among others, the list ``*clgb-list*``, list of user created coalgebras  and reset the global counter to 1.

``(clgb n)`` *[Function]*
> Retrieve in the list ``*clgb-list*`` the coalgebra instance whose identification is $n$. If it does not exist, return ``NIL``.

``(cprd clgb &rest)`` *[Macro]*
> Versatile macro relative to the coproduct of a coalgebra. The first argument is a ``COALGEBRA`` instance. With only one argument, this macro returns the coproduct morphism of the coalgebra.
With more arguments, it applies the coproduct morphism on a combination (``(cprd`` *clgb cmbn*``)``)
or on a pair *degree, generator* (``(cprd ``*clgb degr gnrt*``)``).

``(change-chcm-to-clgb chcm :intr-cprd intr-cprd :cprd-strt cprd-strt :orgn orgn)``
> Build a coalgebra instance from the already created chain complex *chcm*. The user must give via key
parameters a lisp function for the coproduct, the strategy and a comment list.

## Cobar of a coalgebra

Let ${\cal A}$ be a coalgebra, ${\cal M}$ a  right ${\cal A}$-comodule, ${\cal N}$ a left ${\cal A}$-comodule. It means that there exist coproducts $\chi_{\cal M}$ and $\chi_{\cal N}$:

$$\chi_{\cal M}: {\cal M} \longrightarrow {\cal M}\otimes {\cal A},\quad
  \chi_{\cal N}: {\cal N} \longrightarrow {\cal A}\otimes {\cal N}.  $$
  
It is possible to build a chain complex denoted $Cobar^{\cal A}({\cal M}, {\cal N})$, 

$$Cobar^{\cal A}({\cal M}, {\cal N}) = \bigoplus_p
{\cal M}\otimes {\bar{\cal A}}^{\otimes p} \otimes {\cal N}.$$

This section is devoted to the particular case ${\cal M}={\cal N}=\mathbb{Z}$.

So, let ${\cal A}$ be  a coalgebra , assumed 1-connected, i.e. ${\cal A}_1 = 0$ and  ${\cal A}_0 \simeq \mathbb{Z}$, we consider the chain complex, $Cobar^{{\cal A}}(\mathbb{Z}, \mathbb{Z})$, noted more simply  $Cobar({\cal A})$, whose $n$-th component is:

$$ [Cobar({\cal A})]_n = \bigoplus ({\bar{\cal A}}_{i1} \otimes \cdots \otimes {\bar{\cal A}}_{ik}),\quad i_j >0, \quad
i1 + \cdots + ik= n+k.$$

A tensor product $g_1 \otimes \cdots \otimes g_k$, where $g_i$ is a generator of ${\bar{\cal A}}_{i}$ is called, in the software ``Kenzo``, an **algebraic loop**. The reason is the following: if $X$ is a 1-connected simplicial set, we shall see in a next chapter that the program ``Kenzo`` constructs a homotopy equivalence between the chain complex of the loop space ${\cal C}_*(GX)$ and $Cobar({\cal C}_*(X)) = Cobar^{{\cal C}_*(X)}(\mathbb{Z}, \mathbb{Z})$, so that a generator of the last chain complex is a kind of "algebraic" version of a generator of a loop space. The definition of  differential of the Cobar is recalled hereafter.


### Representation of an algebraic loop

An algebraic loop is represented by a lisp object of the form:

> ``(:ALLP``  ($i_1\,.\, g_1$) ... ($i_k\,.\, g_k$)``)``

where the $i_j$ are the degrees *in the cobar chain complex* of the generators $g_j$. In the original coalgebra $\cal A$, the generators $g_j$ had the degree $i_j+1$. The corresponding type is ``ALLP``.
The function to build such an object is also called ``allp``

``(allp degr1 gnr1 degr2 gnr2 ... degrk gnrk)`` *[Function]*
> Construct an algebraic loop, i.e. a tensorial product of degree $\sum degr_i$. The sequence of pairs
$\lbrace degr_i,\,  gnr_i \rbrace$ has an undefinite length and  may be void. In this case,  the algebraic loop is the null algebraic loop, identified in the system by the constant ``+null-allp+``. 
The function ``allp`` accepts also as unique argument a list of the form ``(`` *degr1 gnr1 degr2 gnr2 ... degrk gnrk*``)``

``(allp-p object)`` *[Function]*
> Test if *object* is an algebraic loop.

The associated printing  method prints the object under the form:
> ``<<AlLp [`` $i_1$ $g_1$ ``] ... [`` $i_k$ $g_k$ ``]>>``

The user will have noted that in the Cobar, each generator $g_i$ appears with its own degree in the
underlying chain complex, **lowered by 1**.


### Definition of the chain complex Cobar

The definition of the differential in the Cobar, will be done in three steps. This is better understood, if we consider the following diagram. We recall that the elements of $({\bar{\cal A}}_*^{\otimes_p})_q$ are by convention of degree $q-p$ in the Cobar.

- A chain complex called ``vertical-cobar`` is defined with the *vertical differential* $d_v$. In this case, only the  underlying  chain complex structure is used.
- A horizontal differential $d_h$ is defined. This uses the coproduct structure.
- The final chain complex ``cobar`` is created with differential $d_v+d_h$.

$$
\begin{align}
       &             & \downarrow\quad  &  & \downarrow\qquad & &\\
\cdots & \quad\rightarrow & ({\bar{\cal A}}_*^{\otimes p})_q & \qquad\buildrel {d_h} \over \longrightarrow
                     & ({\bar{\cal A}}_*^{\otimes {p+1}})_q  & \qquad\rightarrow & \cdots \\
       &             &  \downarrow {\scriptstyle d}_v  &  &  \downarrow {\scriptstyle d}_v\quad  & & \\
\cdots & \quad\rightarrow &\quad({\bar{\cal A}}_*^{\otimes p})_{q-1} & \qquad \buildrel {d_h} \over \longrightarrow 
                     & \quad({\bar{\cal A}}_*^{\otimes {p+1}})_{q-1}  & \qquad\rightarrow & \cdots \\ 
       &             & \downarrow\quad  &  & \downarrow\qquad & &
\end{align}
$$

#### Definition of the vertical cobar

To define the vertical cobar chain complex from a chain complex ${\cal C}$, the following functions are provided:

``(cobar-cmpr cmpr)`` *[Function]*
> From the comparison function *cmpr*, build a comparison function for objects of algebraic loop type. 

``(cobar-basis basis)`` *[Function]*
> From the function *basis}* of a  chain complex ${\cal C}$, build a basis function for the vertical cobar chain complex defined on ${\cal C}$. In dimension 0, there is only one basis element, namely the null algebraic loop. If ${\cal C}$ is locally effective, this function returns the symbol ``:locally-effective``.

``(cobar-intr-vrtc-dffr dffr)`` *[Function]*
> From the ``lisp`` function *dffr* of a chain complex  ${\cal C}$, build a lisp function for the differential of the vertical cobar chain complex, according to the formula
$$d_v(g_1 \otimes \cdots \otimes g_n)=
\sum_{i=1}^n{(-1)^{n-i+1+{\sum_{j=1}^{i-1}{\mid g_j \mid}}}
  g_1 \otimes \cdots \otimes g_{i-1} \otimes {\bf dg_i} \otimes g_{i+1} \otimes \cdots \otimes g_n},$$
  where $dg_i$ is the differential of the generator $g_i$ in the original chain complex (function *dffr*) and $\mid g_i \mid$ is the degree of the generator $g_i$ *in the cobar chain complex*.
In general, the differential $dg_i$ is a sum of tensor products. So, applying the distribution law, the right member is a sum of tensor products, represented by a combination.

``(vrtc-cobar chcm)`` *[Method]*
> Build the vertical cobar chain complex with underlying chain complex *chcm*. This is simply done by the following call to ``build-chcm``:
```lisp
(build-chcm
   :cmpr (cobar-cmpr cmpr)
   :basis (cobar-basis basis)
   :bsgn +null-allp+
   :intr-dffr (cobar-intr-vrtc-dffr dffr)
   :strt :gnrt
   :orgn `(vrtc-cobar ,chcm))
```
where, *cmpr*, *basis* and *dffr* are extracted from the chain complex *chcm*. Note that the base generator is the null algebraic loop.

### Example

We  are going to build a coalgebra  which will be reconsidered in a next chapter (Simplicial sets). It is the coalgebra canonicaly associated to the simplicial set $\Delta_2^\mathbb{N}$. This simplicial set is the quotient of $\Delta^\mathbb{N}$ - the standard simplex spanned by the non-negative integers - by the relation identifying, firstly all the vertices with a unique one and then any 1-simplex of $\Delta^\mathbb{N}$ with the unique degeneracy of this unique vertex. In other words, $\Delta_2^\mathbb{N}$ is 1-reduced: 1 vertex, no edges.

Our implementation represents any non--degenerate $n$--simplex as an integer increasing list of length $n+1$. In dimension 0, for every non-negative $m$ and $n$, the vertices ``(m)`` and ``(n)`` are identified; in dimension 1, the list $(m\, n)$ is always an *illegal* 1-simplex. This coalgebra will be located through the symbol ``smp-deltab2``. The lisp definition uses the function ``build-smst`` that will be described in the simplicial set chapter. For the moment, it is sufficient to know, that the function ``build-smst`` allows us to create a coalgebra suitable for the tests. The definition  of $\Delta_2^\mathbb{N}$ given here, is only for pedagogical purpose, an efficient definition using another representation for the simplices is provided in the software ``Kenzo``.

In [ ]:
(def smp-deltab2
    (build-smst
     :cmpr #'(lambda (gsm1 gsm2)
                       (if (rest gsm1) (l-cmpr gsm1 gsm2) :equal))
     :basis :locally-effective
     :bspn '(0)
     :face #'(lambda (i dmn gsm)
                     (case dmn
                         (0 (error "No face in dimension 0"))
                         (1 (error "No non-degenerate simplex in dimension 1"))
                         (2 (absm 1 '(0)))
                         (otherwise (absm 0 (append (subseq gsm 0 i)
                                                    (subseq gsm (1+ i)))))))
     :orgn '(simple-deltab2)))

The following statements show the identification of vertices and that the boundary of a 2-simplex is null.

In [ ]:
(cmpr smp-deltab2 '(5) '(0))

In [ ]:
(? smp-deltab2 2 '(0 1 2))

In [ ]:
(? smp-deltab2 3 '(0 1 2 3))

Let us build the vertical cobar on the coalgebra ``smp-deltab2`` and apply the comparison function and the differential on some algebraic loops.

In [ ]:
(def precobar (vrtc-cobar smp-deltab2))

In [ ]:
(bsgn precobar)

In [ ]:
(def loop-1 (allp 2 '(0 1 2 3)))

In [ ]:
(def loop-2 (allp '(2 (1 2 3 4))))

In [ ]:
(cmpr precobar loop-1 loop-2)

In [ ]:
(? precobar 2 loop-1)

In [ ]:
(? precobar *)

In [ ]:
(def loop-3 (allp 3 '(0 1 2 3 4) 3 '(1 3 5 7 9)))

In [ ]:
(? precobar 6 loop-3)

In [ ]:
(? precobar *)

In [ ]:
(def loop-4 (allp 3 '(0 1 4 5 6) 4 '(2 3 4 5 6 8) 4 '(0 4 5 6 7 8)))

In [ ]:
(? precobar 11 loop-4)

In [ ]:
(? precobar *)

#### Definition of the horizontal differential

So far, the coalgebra structure has not been used but now it will be the main ingredient. If we  make the convention that the elements of $({\bar{\cal A}}_*^{\otimes_p})_q$ are of degree $q-p$, 
we know that exists a canonical "horizontal" differential $d_h$,

$$d_h:({\bar{\cal A}}_*^{\otimes p})_q \longrightarrow ({\bar{\cal A}}_*^{\otimes {p+1}})_q,$$

$$
\begin{align}
d_h( g_1 \otimes \cdots \otimes g_p) = & (-1)^p \Delta g_1 \otimes g_2 \otimes \cdots \otimes g_p \\
                    & + (-1)^{p-1} g_1 \otimes \Delta g_2 \otimes \cdots \otimes g_p \\
                    &  \cdots  \\
                    & - g_1 \otimes g_2 \otimes \cdots \otimes g_{p-1} \otimes \Delta g_p,
\end{align}
$$

where $\Delta$ is the coproduct of the coalgebra. Two functions are designed for that:

``(cobar-intr-hrzn-dffr cprd)`` *[Function]*
> From the coproduct morphism {\em cprd}, build the lisp function with 2 arguments: a degree and an algebraic loop (i.e. a generator) implementing the algorithm for $d_h$.

``(cobar-hrzn-dffr clbg)`` *[Function]*
> Build the horizontal differential morphism, using the slot coproduct *cprd* of
the coalgebra *clbg* and the previous function. Note that when called, this function creates, if not already created, the vertical cobar chain complex on *clbg*:
```lisp
(build-mrph
  :sorc (vrtc-cobar clgb) 
  :trgt (vrtc-cobar clgb) 
  :degr -1
  :intr (cobar-intr-hrzn-dffr cprd) 
  :strt :gnrt
  :orgn `(cobar-hrzn-dffr ,clgb))
```

### Example

We still use our simple coalgebra ``smp-deltab2``. The horizontal differential uses the coproduct of the coalgebra. The following examples show the application of the coproduct (note that there are no 1-simplices in the tensor products). We verify also the property of the horizontal differential.

In [ ]:
(cprd smp-deltab2 2 '(0 1 2))

In [ ]:
(cprd smp-deltab2 3 '(1 2 3 4))

In [ ]:
(cprd smp-deltab2 4 '(0 1 2 3 4))

In [ ]:
(def dh-mrph (cobar-hrzn-dffr smp-deltab2))

In [ ]:
loop-1

In [ ]:
(? dh-mrph 2 loop-1)

In [ ]:
loop-3

In [ ]:
(? dh-mrph 6 loop-3)

In [ ]:
(? dh-mrph *)

In [ ]:
loop-4

In [ ]:
(? dh-mrph 11 loop-4)

In [ ]:
(? dh-mrph *)

#### Final definition of the cobar of a coalgebra

To define completely the cobar chain complex, we have to provide the lisp function for the differential $d_v + d_h$ and, once created the vertical cobar chain complex, to call the building function ``build-chcm`` with adequate arguments.

``(cobar-intr-dffr vrtc-dffr hrzn-dffr)`` *[Function]*
> Define the lisp function for the differential $d_v+d_h$ from the two morphisms *vrtc-dffr* and *hrzn-dffr*. For efficiency reasons, the implementor has chosen to define this new function rather than to use simply the addition of two morphisms.

``(cobar coalgebra)`` *[Method]*
> Build first the vertical cobar chain complex on the coalgebra. Then return a chain complex with the same slots as the vertical cobar chain complex but with new ``:intr-dffr``
and ``:orgn`` slots, as shown in the following definition:

```lisp
(defmethod COBAR ((coalgebra coalgebra))
   (let ((vrtc-cobar (vrtc-cobar coalgebra))
         (cobar-hrzn-dffr (cobar-hrzn-dffr coalgebra)))
        (declare (type chain-complex vrtc-coabr hrzn-cobar))
        (the chain-complex
            (build-chcm
               :cmpr (cmpr vrtc-cobar)
               :basis (basis vrtc-cobar)
               :bsgn +null-allp+
               :intr-dffr (cobar-intr-dffr (dffr vrtc-cobar) 
                                           cobar-hrzn-dffr)
               :strt :gnrt
               :orgn `(add ,vrtc-cobar ,cobar-hrzn-dffr)))))
```

### Example

We may now build the cobar of ``smp-deltab2`` and test the differential.

In [ ]:
(def cobar-deltab2 (cobar smp-deltab2))

In [ ]:
loop-3

In [ ]:
(? cobar-deltab2 6 loop-3)

In [ ]:
(? cobar-deltab2 *)

In [ ]:
loop-4

In [ ]:
(? cobar-deltab2 11 loop-4)

In [ ]:
(? cobar-deltab2 *)

## Other functions for the cobar construction

The vertical cobar construction is *natural* and therefore defines a functor. Consequently, the ``Kenzo`` program contains the following methods.

``(vrtc-cobar mrph)`` *[Method]*
> From the morphism *mrph*, build a new morphism between the vertical-cobar of the source of *mrph* and the vertical cobar of the target of *mrph*, the morphism itself being induced in a natural way by the underlying morphism.

``(vrtc-cobar rdct)`` *[Method]*
> From the reduction *rdct*, build a new reduction, by applying the vertical cobar method to the functions $f$ and $g$. The new homotopy cannot be obtained so simply, being a function of the $f$, $g$ and $h$ of *rdct*. Nevertheless, the source and target of the new homotopy are the vertical cobar of the underlying homotopy.

``(vrtc-cobar hmeq)`` *[Method]*
> Build a homotopy equivalence by applying the vertical cobar construction to the left and right reductions of the homotopy equivalence *hmeq*.

``(cobar hmeq)`` *[Method]*
> Build a  homotopy equivalence where the cobar construction is applied to the underlying homotopy equivalence in the following way. First this construction is limited to the case where the left bottom chain complex of *hmeq* is a **coalgebra**. Then the vertical cobar construction is applied to *hmeq* and the horizontal differential of the left bottom chain complex is propagated upon the new homotopy equivalence with the help of the method ``add``. In other words, starting from the left bottom chain complex, firstly the easy pertubation lemma is applied to obtain a new differential on the top chain complex; then the basic perturbation lemma is applied to the right reduction. If *hmeq* is a
trivial homotopy equivalence, this function returns a trivial one, built on the cobar of the left bottom chain complex of *hmeq*.

### Lisp files concerned in this chapter

``coalgebras.lisp``, ``cobar.lisp``.

[``classes.lisp``, ``macros.lisp``, ``various.lisp``].